# Run the HGT-inference programs 

It is advised to use each program script separately since those are very time consuming. For some, via multiprocessing, we can split up the work to reduce the running time. We are running these in an HPC with ~50 threads each. 

In [2]:
taxonomicID = '1236'
data = '/root/work/projects/paper_comparative_study_of_hgt_inference_methods/data/'

**Note:** 

For each of the scripts run below of the form `run_{inference_program}.py` (e.g. `run_ALE.py`), 
you can use the `-h` flag to see the help message for the script. 

For example, run `python run_ALE.py -h`. 

These scripts have are run with their default parameters. You can take a look at which exact parameters or files are being used by opening the script and looking at the exact `parser.add_argument` calls. If you want to use different files, you can prepare files with the same format as those used in the scripts and pass them as arguments to the scripts.

In [2]:
%%bash
# make directories for programs
mkdir -p program_runs
cd program_runs
mkdir -p ALE AnGST RANGER RANGER-Fast Count GLOOME_with_tree GLOOME_without_tree

## Ranger

From the manual for RANGER-DTL 2.0, the program takes as input a single file (specified using the `–i` command line option) containing first the species tree, followed by a single gene tree. All input trees must be expressed using the Newick format terminated by a semicolon, and they must be fully binary (fully resolved) and rooted. Species names in the species tree must be
unique. E.g., ((speciesA, speciesB), speciesC);

Each leaf in the gene tree must be labeled with the name of the species from which that gene was sampled. If desired, the gene name can be appended to the species name separated by an underscore `_` character. The gene tree may contain any number (zero, one, or more) of homologous genes from the same species. E.g., (((speciesA_gene1, speciesC_gene1), speciesB_geneX), speciesC_gene2);

The following script (`src/run_rangerdtl.py`) runs Ranger on each of the gene trees. First it creates input files for each gene tree in our dataset by concatenating the species tree and the gene tree into one input file. Then it runs using `multiprocessing`, the program `Ranger-DTL-Fast` (from `SupplementaryPrograms/`) and `AggregateRanger` (from `CorePrograms/`) from the dir with Ranger executables. To run `AggregateRanger`, note that the output reconciliation files of Ranger-DTL-Fast needs to be of the format `{prefix}{index}` where in our case the prefix is `recon` and the indices begin with 1. The path to these output files are then provided to `AggregateRanger` by using the prefix only. See the comments in the script for more information.

Run Ranger using the script `run_rangerdtl.py` in the following manner:
```bash
# cd RANGER
$ nohup python ../src/run_rangerdtl.py > nohup_ranger.out & disown
```

### Ranger-Fast

Run Ranger-Fast using the same script but with `--fast` flag:
```bash
# cd RANGER-Fast
$ nohup python ../src/run_rangerdtl.py --fast > nohup_ranger_fast.out & disown
```


## AnGST 

AnGST needs the species tree to be rooted AND to have an additional branch length for the root (prepared in the previous step's notebook; the needed species tree file is named `*_with_root_length.nwk`).

In case of any alteration of the scripts below, note that "AnGST.py" can only be run with python2. 

We need an input file for "AnGST.py". The input file requires to have the path to the species tree/gene tree/output/penalties. Example:

We create the penalty file based on the parameters suggested in the manual.

In [ ]:
%%bash -s "$taxonomicID" "$data"
# create AnGST directory if it doesn't exist
# write penalty file
# these params were taken from the AnGST manual
mkdir -p $2/program_runs/AnGST
cat > ./src/angst_penalty_file.txt << EOL
hgt: 3.0
dup: 2.0
los: 1.0
spc: 0.0
EOL

Run AnGST using the script `run_angst.py` in the following manner:

```bash
$ cd AnGST
# make sure you activate the python2 environment
$ mamba activate hgt_analyses_py2

# run the script
$ nohup ~/mambaforge/envs/hgt_analyses_py2/bin/python ../src/run_angst.py > nohup_run_angst.out & disown
```

## ALE

The script here makes use of the direct compilation of ALE (instead of using `Docker`). Run the script from a terminal, from inside the `ALE` directory inside `program runs` directory. For each gene tree, the script runs a new process where the gene tree is written to a new file, `ALEobserve` is used to create `.ale` files, and `ALEml_undated` is run to perform the reconciliation. Sometimes the ALE programs give errors for no particular reason, so the script also has a number of retries that it can perform for each of the ALE programs before giving up. Use `--help` to see the options available or the default values used.

```bash
# cd ALE
$ nohup python ../src/run_ALE.py > run_ALE_nohup.out & disown
```

## GLOOME and Count

The function below creates the presence-absence (PA) matrix for GLOOME, based on the PA of taxa in the NOGs of interest. We use that matrix (as a fasta file input) along with the species tree (and a separate run, without the tree), to infer gains and losses. Since Count also uses a PA matrix but in TSV format, we prepare it here itself. Note that Count requires each row to be that of a gene family (in our case, a NOG) whereas GLOOME needs each row (aka each sequence in the FASTA file) to be that of a taxon.

### GLOOME ML

In [4]:
%%bash -s "$taxonomicID" "$data"
cd $2/program_runs/
# write param file that uses the rooted tree
cat > ./GLOOME_with_tree/gloome_ml_tree.params << EOL
_seqFile $2$1_pa_matrix.fasta
_treeFile $2$1_wol_tree_pruned_no_internal_labels.nwk
_gainLossDist 1
# for COG and EggNOG only patterns with 3 or more ones are observable
_minNumOfOnes 4
# include Parsimony results along with ML
_costMatrixGainLossRatio 1
## Advanced 
_logValue 4
_outDir Results_GLOOME_ML_with_tree
EOL

# write param file that doesn't use the rooted tree
cat > ./GLOOME_without_tree/gloome_ml_without_tree.params << EOL
_seqFile $2$1_pa_matrix.fasta
_gainLossDist 1
# for COG and EggNOG only patterns with 3 or more ones are observable
_minNumOfOnes 4
# include Parsimony results along with ML
_costMatrixGainLossRatio 1
## Advanced
_logValue 4
_outDir Results_GLOOME_ML_without_tree
EOL

Run Gloome via command line for each of the param files as input, inside GLOOME directory (`cd GLOOME`). Using `nohup`, `&`, and `disown` here because the process may take a while to finish, and I am running it remotely on a server. Processes shouldn't be affected this way, even if I logout.

```bash
nohup ~/bin/GLOOME.VR01.266 gloome_tree.params > nohup_gloome_run_with_tree.out & disown

nohup ~/bin/GLOOME.VR01.266 gloome_without_tree.params > nohup_gloome_run_without_tree.out & disown
```


### GLOOME MP

In [15]:
%%bash -s "$taxonomicID" "$data"
# we need a params file for each iteration of the _costMatrixGainLossRatio
# _costMatrixGainLossRatio is the ratio of the cost of a gain to the cost of a loss
# the default is 1:1 but we want to test 0.33, 0.5, 1, 2, ..., 8
# for each of these ratios we need to create a params file
cd $2/program_runs/
for costRatio in 0.33 0.5 1 2 3 4 5 6 7 8; do
# write param file that uses the rooted tree
cat > ./GLOOME_with_tree/gloome_tree_${costRatio}.params << EOL
_seqFile $2$1_pa_matrix.fasta
_treeFile $2$1_wol_tree_pruned_no_internal_labels.nwk
_costMatrixGainLossRatio $costRatio
_logValue 4
_performOptimizations 0
_outDir Results_GLOOME_MP_with_tree_$costRatio
EOL

# write param file that doesn't use the rooted tree
cat > ./GLOOME_without_tree/gloome_without_tree_${costRatio}.params << EOL
_seqFile $2$1_pa_matrix.fasta
_costMatrixGainLossRatio $costRatio
_logValue 4
_performOptimizations 0
_outDir Results_GLOOME_MP_without_tree_$costRatio
EOL
done


Then run all of the GLOOME MP scripts like so:
```bash
cd GLOOME_with_tree
for i in 0.33 0.5 1 2 3 4 5 6 7 8 ; do nohup ~/bin/GLOOME.VR01.266 gloome_tree_${i}.params > nohup_gloome_run_with_tree_${i}.out & disown ; done
```

```bash
cd ../GLOOME_without_tree
for i in 0.33 0.5 1 2 3 4 5 6 7 8 ; do nohup ~/bin/GLOOME.VR01.266 gloome_without_tree_${i}.params > nohup_gloome_run_without_tree_${i}.out & disown ; done
```

### Count ML

```bash
# cd Count/Count_ML
nohup java -Xmx2048M -cp ~/bin/Count/Count.jar ca.umontreal.iro.evolution.genecontent.ML -v true  ../../../1236_wol_tree_pruned_with_internal_labels.nwk ../1236_pa_matrix.tsv > 1236_rates.r & disown
# and after completing the above:
nohup java -Xmx2048M -cp ~/bin/Count/Count.jar ca.umontreal.iro.evolution.genecontent.Posteriors ../../../1236_wol_tree_pruned_with_internal_labels.nwk ../1236_pa_matrix.tsv ./1236_rates.r  > Count_output.tsv & disown
```

### Count MP

In [7]:
%%bash -s "$taxonomicID" "$data"
# exit if command exits with non-zero status
set -e
# print working directory
echo "Current working directory: $(pwd)"
# change to the directory where Count will be run
cd ./program_runs/Count/Count_MP/

# for `gain` parameter being 0.33, 0.5, 1, 2, ,.. 8, run Count
for g in 0.33 0.5 1 2 3 4 5 6 7 8;
do
    # run Count
    java -Xmx2048M -cp ~/bin/Count/Count.jar ca.umontreal.iro.evolution.genecontent.AsymmetricWagner -gain $g $2$1_wol_tree_pruned_with_internal_labels.nwk $2$1_pa_matrix.tsv > ${1}_Count_output_gain_${g}.tsv && echo "Count with gain ratio $g done" &&
    # grep the output
    grep "# PRESENT" ${1}_Count_output_gain_${g}.tsv > ${1}_Count_output_gain_${g}_genome_sizes.tsv &&
    grep "# CHANGE" ${1}_Count_output_gain_${g}.tsv > ${1}_Count_output_gain_${g}_changes.tsv &&
    grep "# FAMILY" ${1}_Count_output_gain_${g}.tsv > ${1}_Count_output_gain_${g}_families.tsv &&
    echo "Count output for gain ratio $g grepped"
done


Current working directory: /root/work/projects/paper_comparative_study_of_hgt_inference_methods/02-run_programs
Count with gain ratio 0.33 done
Count output for gain ratio 0.33 grepped
Count with gain ratio 0.5 done
Count output for gain ratio 0.5 grepped
Count with gain ratio 1 done
Count output for gain ratio 1 grepped
Count with gain ratio 2 done
Count output for gain ratio 2 grepped
Count with gain ratio 3 done
Count output for gain ratio 3 grepped
Count with gain ratio 4 done
Count output for gain ratio 4 grepped
Count with gain ratio 5 done
Count output for gain ratio 5 grepped
Count with gain ratio 6 done
Count output for gain ratio 6 grepped
Count with gain ratio 7 done
Count output for gain ratio 7 grepped
Count with gain ratio 8 done
Count output for gain ratio 8 grepped


## Wn

Wn can be run directly using the script `run_Wn.py` in the `src` directory:

```bash
cd 02-run_programs/program_runs/Wn/
nohup ~/mambaforge/envs/hgt_analyses/bin/python ../../../02-run_programs/src/run_Wn.py > nohup_run_Wn.log & disown
```

The results will be saved by default in the `Results` directory. 
